## Analysing H1B Visa data Trends

H1B visa is a nonimmigrant visa issued to gradaute level applicants allowing them to work in the United States. The employer sponsors the H1B visa for workers with theoretical or technical expertise in specialized fields such as in IT, finance, accounting etc. An interesting fact about immigrant workers is that about 52 percent of new Silicon valley companies were founded by such workers during 1995 and 2005. Some famous CEOs like Indira Nooyi (Pepsico), Elon Musk (Tesla), Sundar Pichai (Google),Satya Nadella (Microsoft) once arrived to the US on a H1B visa.\
**Motivation**: Our team consists of five international graduate students, in the future we will be applying for H1B visa. The visa application process seems very long, complicated and uncertain. So we decided to understand this process and use Machine learning algorithms to predict the acceptance rate and trends of H1B visa.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install autocorrect
import pandas as pd
import numpy as np
import nltk,warnings
import clean_wage as cw
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statistics import mean
from autocorrect import Speller 
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder #ONE HOT ENCODING
from sklearn.ensemble import RandomForestClassifier #Build model - Random Forest Classifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
nltk.download('wordnet')
nltk.download('words')

### Data 
The data used in the project has been collected from <a href="https://www.foreignlaborcert.doleta.gov/performancedata.cfm">the Office of Foreign Labor Certification (OFLC).</a>The Data provides insight into each petition with information such as the Job title, Wage, Employer, Worksite location etc. To get the dataset click on the above link-> click on Disclosure data -> scroll down to H1B data.\


In [0]:
file2015=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY15_Q4.csv',encoding='latin-1', low_memory=False)
file2015.rename(columns = {'H-1B_DEPENDENT':'H1B_DEPENDENT','SOC_NAME':'SOC_TITLE','WAGE_RATE_OF_PAY':'WAGE_RATE_OF_PAY_FROM'}, inplace = True)
df2015=file2015[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]
df2015['WAGE_RATE_OF_PAY_FROM']=df2015['WAGE_RATE_OF_PAY_FROM'].str.split('-').str[0]

In [0]:
file2016=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY16.csv',encoding='latin-1', low_memory=False)
file2016.rename(columns = {'H-1B_DEPENDENT':'H1B_DEPENDENT','SOC_NAME':'SOC_TITLE'}, inplace = True)
df2016=file2016[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
file2017=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY17.csv',encoding='latin-1', low_memory=False)
file2017.rename(columns = {'SOC_NAME':'SOC_TITLE'}, inplace = True)
df2017=file2017[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
file2018=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY2018_EOY.csv',encoding='latin-1', low_memory=False)
file2018.rename(columns = {'SOC_NAME':'SOC_TITLE'}, inplace = True)
df2018=file2018[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
file2019=pd.read_csv('/content/gdrive/My Drive/H1B_project/H-1B_Disclosure_Data_FY2019.csv',encoding='latin-1', low_memory=False)
file2019.rename(columns = {'H-1B_DEPENDENT':'H1B_DEPENDENT','WAGE_RATE_OF_PAY_FROM_1':'WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY_1':'WAGE_UNIT_OF_PAY',\
                           'WORKSITE_CITY_1':'WORKSITE_CITY','WORKSITE_STATE_1':'WORKSITE_STATE'}, inplace = True)
df2019=file2019[['CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED', 'DECISION_DATE',
       'VISA_CLASS','FULL_TIME_POSITION','JOB_TITLE', 'SOC_CODE', 'SOC_TITLE','EMPLOYER_NAME','WAGE_RATE_OF_PAY_FROM',
       'WAGE_UNIT_OF_PAY','WORKSITE_CITY','WORKSITE_STATE','H1B_DEPENDENT']]

In [0]:
cleaned=pd.concat([df2015,df2016,df2017,df2018,df2019])
cleaned.shape

In [0]:
del file2015 # To avoid using the RAM completely
del file2016
del file2017
del file2018
del file2019
del df2019
del df2018
del df2017
del df2016
del df2015

### Exploratory Data Analysis
Before we begin working on our data we need to understand the traits of our data which we accomplish using EDA. We see that we have about 260 columns , not all 260 columns have essential information that contributes to our analysis. Hence we pick out the columns such as case status( Accepted/ Denied) ,Employer, Job title etc.We can look at all the colums and the types of object in each column using cleaned.info(). We also use cleaned['column_name'].value_counts() to find the classes in the column along with the count. For feature engineering we are converting quantitative data to categorical.


####Granularity:
Each row in the dataframe represents an application that was filled by the employer and some values are added during the processing of the application such as DECISION_DATE, CASE_STATUS.

In [0]:
cleaned.dropna(inplace=True)
cleaned.shape

In [0]:
cleaned.head(3)

In [0]:
cleaned['VISA_CLASS'].value_counts()

In [0]:
cleaned.drop(labels=cleaned.loc[cleaned['VISA_CLASS']!='H-1B'].index , inplace=True)

In [0]:
cleaned['CASE_STATUS'].value_counts()

In [0]:
cleaned.drop(labels=cleaned.loc[cleaned['CASE_STATUS']=='WITHDRAWN'].index , inplace=True)
cleaned.drop(labels=cleaned.loc[cleaned['CASE_STATUS']=='CERTIFIED-WITHDRAWN'].index , inplace=True)
cleaned['CASE_STATUS'].value_counts()

In [0]:
cleaned.info()

In [0]:
cleaned['WAGE_RATE_OF_PAY_FROM'].apply(type).value_counts()

####Temporality
The data is available from 

In [0]:
max_date=cleaned.CASE_SUBMITTED.max()	
min_date=cleaned.CASE_SUBMITTED.min()
print("The date in the dataset ranges from "+str(max_date)+" to",min_date)

In [0]:
cleaned['WAGES']=cleaned['WAGE_RATE_OF_PAY_FROM'].apply(cw.clean_wages).astype('float')

In [0]:
cleaned['WAGE_UNIT_OF_PAY'].value_counts()

In [0]:
cleaned= cw.clean_wageUnit(np,cleaned)
cleaned.drop(columns=['WAGE_RATE_OF_PAY_FROM','WAGE_UNIT_OF_PAY'],axis=1,inplace=True)

In [0]:
print(cleaned['WAGES'].describe())
sns.boxplot(y=cleaned['WAGES'])
plt.show()

We can see that we have large outliers , we can remove them from the data. 

In [0]:
# Remove the outliers.
lowerBound = 0.001
upperBound = 0.95
result = cleaned.WAGES.quantile([lowerBound, upperBound])
cleaned = cleaned[(result.loc[lowerBound] < cleaned.WAGES.values) &\
                  (cleaned.WAGES.values < result.loc[upperBound])]

print(cleaned['WAGES'].describe())
sns.boxplot(y=cleaned['WAGES'])
plt.show()

In [0]:
Top_Employer=cleaned['EMPLOYER_NAME'].value_counts()[:10]
plt.figure(figsize=[8,8])
ax=sns.barplot(y=Top_Employer.index,x=Top_Employer.values,palette=sns.color_palette('viridis',10))
ax.tick_params(labelsize=12)
for i, v in enumerate(Top_Employer.values): 
    ax.text(.5, i, v,fontsize=15,color='white',weight='bold')
plt.title('Top 10 Companies sponsoring H1B Visa in 2019', fontsize=20)
plt.show()

In [0]:
cleaned.WORKSITE_STATE.value_counts()

In [0]:
cleaned= cw.clean_states(cleaned)

#### Data Scope
The data available contains applications submitted by Employers from 2015 till 2019. \
Geographical Scope: The WORKSITE_STATE gives information about the state where the primary worksite location. The data set covers all the states of US. 
The Data set also cover different Industries majority of the applications are for IT industry but not limited to IT, it also covers Medical, Pharmacy, Law, Carpentery etc.

In [0]:
cleaned.SOC_TITLE.value_counts()

In [0]:
cleaned.WORKSITE_STATE.value_counts()

####Faithfulness:
Each record in the dataframe represents an application filled by the employer, hence we see a lot of spelling mistakes and some punctutaions like ,(comma) and some numbers in the JOB_TITLE, SOC_TITLE feilds, but there are no inconsistencies and data falsification.

In [0]:
#The Job_tite and Soc_title had lot numbers in between and ,(comma) so it has to be cleaned
cleaned['JOB_TITLE']=cleaned.JOB_TITLE.apply(lambda txt: " ".join([cw.remove_num(i) for i in txt.lower().split()]))
cleaned['JOB_TITLE']=cleaned['JOB_TITLE'].str.replace(',', '')

cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([cw.remove_num(i) for i in txt.lower().split()]))
cleaned['SOC_TITLE']=cleaned['SOC_TITLE'].str.replace(',', '')
cleaned['JOB_TITLE'].value_counts()

In [0]:
#Function to clean data columns, to correct spelling mistakes and convert plural words to singular
lemmatizer = nltk.stem.WordNetLemmatizer()
words = set(nltk.corpus.words.words())
spell = Speller()
def lemmatize_text(text):
     return lemmatizer.lemmatize(text)
def spelling_checker(text):
     return spell(text)

In [0]:
cleaned['JOB_TITLE']=cleaned.JOB_TITLE.apply(lambda txt: " ".join([lemmatize_text(i) for i in txt.lower().split()]))
cleaned['JOB_TITLE']=cleaned.JOB_TITLE.apply(lambda txt: " ".join([spelling_checker(i) for i in txt.lower().split()]))

In [0]:
cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([lemmatize_text(i) for i in txt.lower().split()]))
cleaned['SOC_TITLE']=cleaned.SOC_TITLE.apply(lambda txt: " ".join([spelling_checker(i) for i in txt.lower().split()]))

The has lot of different subcategories which have less than 15 applications in them which does not show much significance , removing them help us condense the data and take only the significant rows.


In [0]:
cleaned= cw.drop_less_significant(cleaned)